### Workflow for getting composition for each roi annotation for each DLmask (DLmask generated from entire mask of ndpi):

In [2]:
OPENSLIDE_PATH = r'C:\Users\Kevin\Desktop\openslide-win64-20221217\bin'

import os
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

In [3]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import os
import cv2
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import openslide
from PIL import Image
from matplotlib import pyplot as plt
import skimage.measure
import matplotlib.pyplot as plt
import scipy.stats as stats
import torchvision.transforms as transforms
from glob import glob
from time import time
from skimage.measure import label

In [5]:
# quick pandas work to see if ndpi's in image list have the DL segmented masks!
# read xml files for roi annotations
roi_src = r'\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi'
# read DL mask files for DL segmented masks
dl_src = r'\\shelter\Kyu\skin_aging\clue_cohort\DLmask1um'
# read image list and fill in DL_segmented column (0=DLmask missing, 1=DLmask exists)
im_list = r'\\shelter\Kyu\skin_aging\clue_cohort\CLUE_image_list_230207.xlsx'

In [7]:
roi_list = [x for x in os.listdir(roi_src) if x.endswith('.xml')] #301 rows
dl_list = [x for x in os.listdir(dl_src) if x.endswith('.tif')] #301 rows
roi_complete_path = [os.path.join(x,roi_src) for x in roi_list]
dl_complete_path = [os.path.join(x,dl_src) for x in dl_list]
roi_raw = [x.replace(".xml","") for x in roi_list]
dl_raw = [x.replace(".tif","") for x in dl_list]

In [4]:
# image_list_csv_raw = pd.read_excel(r'\\shelter\Kyu\skin_aging\clue_cohort\CLUE_image_list_230207.xlsx')

In [ ]:
# dl_list = [x.replace(".tif","") for x in dl_list]

In [5]:
# def check_DLexist(row):
#     row = row.replace('.ndpi','')
#     return row in dl_list

In [6]:
# image_list_csv_raw['DL_segmented'] = image_list_csv_raw.apply(lambda row : check_DLexist(row['filename']), axis = 1)

In [8]:
# image_list_csv_raw.head()

,index,gender,age,pid,path,filename,year,sample id,label name,sub label,...,Inbal comment,ROI ANNOTATED (0 :not annotated 1.annotated),roi reviewed,Nroi,"hypodermis (1=yes, 0=no)",has hair,DV200,use_visium,DL_annotated,DL_segmented
0,1,NaN,NaN,1,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-10 17.43.31.ndpi,2,15926,02-15926,1,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,False
1,2,NaN,NaN,1,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-10 20.49.53.ndpi,2,15926,02-15926,2,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,False
2,3,NaN,NaN,2,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-10 17.55.47.ndpi,2,16107,02-16107,FS,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,False
3,4,NaN,NaN,2,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-08 15.45.22.ndpi,2,16107,02-16107,FS,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,False
4,5,M,55.646575,3,\\fatherserverdw\kyuex\clue images\2022-06-30 ...,2022-06-30 17.54.56.ndpi,2,16145,02-16145,A2,...,NaN,1,1.0,NaN,1,NaN,NaN,NaN,NaN,False


In [16]:
# image_list_csv_raw.to_excel(r'\\shelter\Kyu\skin_aging\clue_cohort\CLUE_image_list_230207_v2.xlsx')

In [ ]:
# First, input xml_filepath and output a dataframe of X,Y coordinates in general. (can be used for ROI as well)
def xml_to_df(xml_filepath):
    tree = ET.parse(xml_filepath)
    root = tree.getroot()
    append_df = []
    for index, Annotation in enumerate(root.iter("Annotation")):
        for Region in Annotation.iter('Region'):
            x = np.array([Vertex.get('X') for Vertex in Region.iter('Vertex')])
            y = np.array([Vertex.get('Y') for Vertex in Region.iter('Vertex')])
            id = np.array([int(Region.get('Id'))])
            classnames = index + 1
            coord_dict = {"ClassNames": [classnames], "X": [x], "Y": [y], "ID": [id]}
            df = pd.DataFrame(data = coord_dict)
            df.ID = df.ID.astype(int)
            append_df.append(df)
    coord_df = pd.concat(append_df).reset_index(drop=True)
    return(coord_df)

In [ ]:
# read a first DLmask and its corresponding roi xml file

roi_png_dst = os.path.join(roi_src,'roi_png_1x')
DLmasks =
for DLmask in DLmasks:
    DLarr = Image.open(DLmask)
    roi_fn = DLmask.replace("png","xml")
    roi_xml = xml_to_df(os.path.join(roi_src,roi_fn))
    roi_arr = df2png(roi_xml) #for example: "\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeledmask_20rsf"
    # roi_png is an image with grayscale labeled mask, save this.
    roi_png = Image.fromarray(roi_arr)
    roi_png.save(os.path.join(roi_png_dst,roi_fn.replace("xml","png")))
    for sectionID in range(1,np.max(roi_png)):
        DL_roi = DLarr[roi_png==sectionID]
        composition = dl2comp(DL_roi)
        comps.append(compositions)


# output composition data by roi
xl_dst = r"\\shelter\Kyu\skin_aging\clue_cohort\CLUE_morphometrics_230207.xlsx"